<a href="https://colab.research.google.com/github/juanescendales/IAIdentifiacionDeColoresFlores/blob/master/MicroProyecto2_ClusteringCultivoFlores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identificación de color en imágenes de cultivos de flores


Se quiere determinar por medio de clustering los tipos de cultivos flores visibles e
indicar el porcentaje cultivado, realizando la mayor caracterización de los datos posible. Esto con un data-set de dos imagenes



# Preprocesamiento de los datos